In [3]:
import numpy as np
import pandas as pd
import re
import nltk
import string

pd.options.mode.chained_assignment = None

# from nltk.tokenize import word_tokenize
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from googletrans import Translator
pd.set_option('display.max_colwidth', None)

import warnings
warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv('../Labelling/Labelled_dataset.csv')
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,English,Sentiment,Composes
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,"Indonesia Recession, Entrepreneur Says New Economic Growth Will Rise in 2023 https://t.co/Xp2EtHsrKP",Negative,"{'neg': 0.182, 'neu': 0.649, 'pos': 0.169, 'compound': -0.0516}"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,"If it's cancer, it looks like it's already entered stage four, it's already rotting, waiting to collapse like a dog with diarrhea that has lost its strength.\nThis is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult... https://t.co/780pIX3sf3",Negative,"{'neg': 0.268, 'neu': 0.608, 'pos': 0.125, 'compound': -0.8558}"
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,"This is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult to avoid https://t.co/fvPodFfNua",Negative,"{'neg': 0.331, 'neu': 0.669, 'pos': 0.0, 'compound': -0.8362}"


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2299 entries, 0 to 2298
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Datetime   2299 non-null   object
 1   Tweet Id   2299 non-null   int64 
 2   Tweet      2299 non-null   object
 3   Username   2299 non-null   object
 4   Language   2299 non-null   object
 5   English    2299 non-null   object
 6   Sentiment  2299 non-null   object
 7   Composes   2299 non-null   object
dtypes: int64(1), object(7)
memory usage: 143.8+ KB


# Cleansing Text

### Function

In [6]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', str(text)) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', str(text)) # remove hashtag
    text = re.sub(r'RT[\s]', '', str(text)) # remove RT
    text = re.sub(r"http\S+", '', str(text)) # remove link
    text = re.sub(r"https\S+", '', str(text)) # remove link
    text = re.sub(r'[0-9]+', '', str(text)) # remove numbers
    text = re.sub(r'(.)\1+', r'\1', str(text))


    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text

    return text

def casefoldingText(text): 
    text = text.lower() 
    return text

def tokenizingText(text): 
    text = word_tokenize(text) 
    return text

def acr(text):
    acr_dict = {
        'ri': 'indonesia',
        'bi': 'bank indonesia',
        'bbm' : 'bahan bakar minyak',
        'umkm' : 'usaha mikro kecil menengah',
        'menparekraf' : 'menteri pariwisita dan ekonomi kreatif',
        'cryto' : 'crypto',
        'menkeu' : 'menteri keuangan',
        'serem' : 'seram',
        'ojk' : 'otoritas jasa keungan',
        'gw' : 'saya',
        'bgt' : 'sangat',
        'mang' : 'memang',
        'cth' : 'contoh',
        'kominfo' : 'kementrian komunikasi dan informatika republik indonesia',
        'bumd' : 'badan usaha milik daearah',
        'bumn' : 'badan usaha milik negara'
    }

    text_words = text.split()
    for i, word in enumerate(text_words):
        if word.lower() in acr_dict:
            text_words[i] = acr_dict[word]
    return ' '.join(text_words)

def filteringText(text): # Remove stopwors in a text
    set_stop = stopwords.words('indonesian')
    set_stop.extend(["yg","dg","rt","dgn","ny","d",'klo', 'klu', 'adaekonomi', 'ad', 'adain', 'adb', 'al', 'ampdipuji', 'an', 'anieshrsfpi', 'arsjad', 'asf', 'askrl', 'bmw', 'blok', 'bni', 'bp', 'bpa', 
                     'bpjs', 'bri', 'brimo', 'btpn', 'buahahahahahahahahahahahahahahahahahahahahahahahahahahaha', 'betmen', 'bgd', 'bgmana', 'bgmn', 'bgnya', 
                     'awowkwowkwok', 'bagimn', 'anies', 'baswedan', 'aamin', 'abamg', 'abcde', 'abar', 'absorber', 'acep', 'leslar', 'hantu', 'gagak', 'anjir', 'erick', 'thohir', 'ceo', 'jpmorgan' ,
                     'chase', 'jamie', 'dimon', 'tv', 'yaa', 'allah', 'yesus', 'mmc', 'kaffah', 'svt', 'gais', 'skz','amp', 'belon', 'kalo', 'biar', 'lg', 'bikin', 'sri', 'anjing', 'layof', 
                     'jokowi', 'luhut', 'mulyani', 'herry', 'niagahoster', 'by', 'thn', 'kanker','bilang','gak', 'ga', 'krn', 'nya', 'nih', 'sih','si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'jd', 'jgn', 'sdh', 'aja', 'n', 'djp', 'sunarya', 't','nyg', 
                     'hehe', 'pen', 'u', 'nan', 'loh','rt','&amp', 'yah', 'byk', 'byr', 'cbi', 'cc', 'ce', 'cebong', 'diy', 'dki', 'dkt', 'dm', 'drs', 'dr', 'prof', 'elon', 'musk', 'yth', 'zero', 'zom', 'zh', 'ysl', 'yoy', 'yra', 'yorobun', 'ygy', 'yg', 'yl', 'yo'])
    listStopwords = set(set_stop)
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    return text

def stemmingText(text): 
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): 
    sentence = ' '.join(word for word in list_words)
    return sentence

### Calling Function

In [7]:
df['Tweet_clean'] = df['Tweet'].apply(cleaningText)
df['Tweet_clean'] = df['Tweet_clean'].apply(casefoldingText)

df.head()

,Datetime,Tweet Id,Tweet,Username,Language,English,Sentiment,Composes,Tweet_clean
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,"Indonesia Recession, Entrepreneur Says New Economic Growth Will Rise in 2023 https://t.co/Xp2EtHsrKP",Negative,"{'neg': 0.182, 'neu': 0.649, 'pos': 0.169, 'compound': -0.0516}",indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,"If it's cancer, it looks like it's already entered stage four, it's already rotting, waiting to collapse like a dog with diarrhea that has lost its strength.\nThis is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult... https://t.co/780pIX3sf3",Negative,"{'neg': 0.268, 'neu': 0.608, 'pos': 0.125, 'compound': -0.8558}",kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,"This is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult to avoid https://t.co/fvPodFfNua",Negative,"{'neg': 0.331, 'neu': 0.669, 'pos': 0.0, 'compound': -0.8362}",ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari


In [8]:
df['Tweet_clean'] = df['Tweet_clean'].apply(acr)

df.head()

,Datetime,Tweet Id,Tweet,Username,Language,English,Sentiment,Composes,Tweet_clean
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,"Indonesia Recession, Entrepreneur Says New Economic Growth Will Rise in 2023 https://t.co/Xp2EtHsrKP",Negative,"{'neg': 0.182, 'neu': 0.649, 'pos': 0.169, 'compound': -0.0516}",indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,"If it's cancer, it looks like it's already entered stage four, it's already rotting, waiting to collapse like a dog with diarrhea that has lost its strength.\nThis is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult... https://t.co/780pIX3sf3",Negative,"{'neg': 0.268, 'neu': 0.608, 'pos': 0.125, 'compound': -0.8558}",kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit Dihindari https://t.co/fvPodFfNua",BizlawIndonesia,in,"This is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult to avoid https://t.co/fvPodFfNua",Negative,"{'neg': 0.331, 'neu': 0.669, 'pos': 0.0, 'compound': -0.8362}",ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit dihindari


In [9]:
df['Tweet_preprocessed'] = df['Tweet_clean'].apply(tokenizingText)
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,English,Sentiment,Composes,Tweet_clean,Tweet_preprocessed
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,"Indonesia Recession, Entrepreneur Says New Economic Growth Will Rise in 2023 https://t.co/Xp2EtHsrKP",Negative,"{'neg': 0.182, 'neu': 0.649, 'pos': 0.169, 'compound': -0.0516}",indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, pengusaha, sebut, pertumbuhan, ekonomi, baru, bangkit, di]"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, mengatakan, pertumbuhan, ekonomi, baru, bisa, normal, pada, namun, jika, tidak, ada, perbaikan, dalam, penanganan, pandemi, dan, resesi, ekonomi, terus, berlanjut, hinga, indonesia, kemungkinan, akan, masuk, ke, jurang, depresi, ekonomi]"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, mengatakan, pertumbuhan, ekonomi, baru, bisa, normal, pada, namun, jika, tidak, ada, perbaikan, dalam, penanganan, pandemi, dan, resesi, ekonomi, terus, berlanjut, hinga, indonesia, kemungkinan, akan, masuk, ke, jurang, depresi, ekonomi]"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,"If it's cancer, it looks like it's already entered stage four, it's already rotting, waiting to collapse like a dog with diarrhea that has lost its strength.\nThis is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult... https://t.co/780pIX3sf3",Negative,"{'neg': 0.268, 'neu': 0.608, 'pos': 0.125, 'compound': -0.8558}",kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[kalau, kanker, sepertinya, sudah, masuk, stadium, empat, sudah, keropos, menungu, ambruk, bagai

In [10]:
df['Tweet_preprocessed'] = df['Tweet_preprocessed'].apply(filteringText)
df.head()

,Datetime,Tweet Id,Tweet,Username,Language,English,Sentiment,Composes,Tweet_clean,Tweet_preprocessed
0,2020-11-05 08:25:32+00:00,1324266586883026945,"Indonesia Resesi, Pengusaha Sebut Pertumbuhan Ekonomi Baru Bangkit di 2023 https://t.co/Xp2EtHsrKP",liputan6dotcom,in,"Indonesia Recession, Entrepreneur Says New Economic Growth Will Rise in 2023 https://t.co/Xp2EtHsrKP",Negative,"{'neg': 0.182, 'neu': 0.649, 'pos': 0.169, 'compound': -0.0516}",indonesia resesi pengusaha sebut pertumbuhan ekonomi baru bangkit di,"[indonesia, resesi, pengusaha, pertumbuhan, ekonomi, bangkit]"
1,2020-10-17 10:36:09+00:00,1317414089144987649,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, pertumbuhan, ekonomi, normal, perbaikan, penanganan, pandemi, resesi, ekonomi, berlanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
2,2020-09-28 05:23:24+00:00,1310450011230990342,"Pengamat mengatakan, pertumbuhan ekonomi baru bisa normal pada 2023. Namun, jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hingga 2021, Indonesia kemungkinan akan masuk ke jurang depresi ekonomi.\n\n#WaktunyaLokadata https://t.co/AwZlqdViU8",LokadataID,in,"Observers say that economic growth will only return to normal in 2023. However, if there is no improvement in handling the pandemic and the economic recession continues until 2021, Indonesia is likely to enter the brink of economic depression.\n\n#TimeLokadata https://t.co/AwZlqdViU8",Negative,"{'neg': 0.185, 'neu': 0.762, 'pos': 0.054, 'compound': -0.7491}",pengamat mengatakan pertumbuhan ekonomi baru bisa normal pada namun jika tidak ada perbaikan dalam penanganan pandemi dan resesi ekonomi terus berlanjut hinga indonesia kemungkinan akan masuk ke jurang depresi ekonomi,"[pengamat, pertumbuhan, ekonomi, normal, perbaikan, penanganan, pandemi, resesi, ekonomi, berlanjut, hinga, indonesia, masuk, jurang, depresi, ekonomi]"
3,2020-09-24 13:42:22+00:00,1309126027755556864,"Kalau Kanker, Sepertinya Sudah Masuk Stadium Empat, Sudah Keropos, Menunggu Ambruk Bagai Anjing Mencret yg Sudah Kehilangan Tenaga.\nIni Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Massal Sulit D... https://t.co/780pIX3sf3",myputun,in,"If it's cancer, it looks like it's already entered stage four, it's already rotting, waiting to collapse like a dog with diarrhea that has lost its strength.\nThis is not good news, the Indonesian economy is predicted to only recover from recession in 2023, mass bankruptcy is difficult... https://t.co/780pIX3sf3",Negative,"{'neg': 0.268, 'neu': 0.608, 'pos': 0.125, 'compound': -0.8558}",kalau kanker sepertinya sudah masuk stadium empat sudah keropos menungu ambruk bagai anjing mencret yg sudah kehilangan tenaga ini bukan kabar baik ekonomi indonesia diprediksi baru sembuh dari resesi pada kebangkrutan masal sulit d,"[masuk, stadium, keropos, menungu, ambruk, mencret, kehilangan, tenaga, kabar, ekonomi, indonesia, diprediksi, sembuh, resesi, kebangkrutan, masal, sulit]"
4,2020-09-24 07:31:13+00:00,1309032626439757826,"Ini Bukan Kabar Baik, Ekonomi Indonesia Diprediksi Baru Sembuh dari Resesi pada 2023, Kebangkrutan Mas

In [11]:
df['Tweet_preprocessed'] = df['Tweet_preprocessed'].apply(stemmingText)
df.head()

In [ ]:
df['Clean_Sentence'] = df['Tweet_preprocessed'].apply(toSentence)

df.head()

## Remove Duplicate

In [ ]:
df.duplicated(subset='Clean_Sentence', keep='first').sum()  # count duplicated

In [ ]:
df.drop_duplicates(subset='Clean_Sentence', inplace=True) # remove duplicated

In [ ]:
print(df.info(), df.duplicated(subset='Clean_Sentence', keep='first').sum())